In [30]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Define Foursquare Credentials and Version

In [38]:
# @hidden_cell
CLIENT_ID = 'V53MHPBBL23EVUFTF31HTKJAFRJIK2QL1WJURTZ3ARRFF3KE' # Foursquare ID
CLIENT_SECRET = 'G5Z0ALDAAKEOPESPXKBZPXNB2TIEJKGWM33ZGSBCNW0VGW0T' # Foursquare Secret
VERSION = '20180604'
LIMIT = 100

#### Explore venues around my work place 

In [25]:
address = 'Russia, Taganrog'
#address = 'Россия, Таганрог, Греческая 104'
#address = 'Russia, Taganrog, Grecheskaya 104'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.2153657 38.9285216


In [49]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            500, 
            LIMIT)

In [33]:
import json, codecs
def json_to_file(data, name='work_nbrs.json'):
    with open(name, 'wb') as f:
        json.dump(data, codecs.getwriter('utf-8')(f), ensure_ascii=False)

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Neighborhood', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)
        

In [53]:
venues = getNearbyVenues(
    names=['Grecheskaya'],
    latitudes=[47.21724655],
    longitudes=[38.9282073366964],
    radius=10000)

In [55]:
venues.head()

,Neighborhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Grecheskaya,47.217247,38.928207,Мясник и бык,47.212617,38.933402,Theme Restaurant
1,Grecheskaya,47.217247,38.928207,Театр имени А. П. Чехова,47.216325,38.928217,Theater
2,Grecheskaya,47.217247,38.928207,Таганрогский художественный музей,47.210121,38.930846,Art Museum
3,Grecheskaya,47.217247,38.928207,Три Орешка,47.215685,38.931146,Coffee Shop
4,Grecheskaya,47.217247,38.928207,Рашен Страшен,47.218004,38.916598,Cocktail Bar


#### Searching all venues in the Tagarog city

In [49]:
address = 'Russia, Taganrog'
loc = geolocator.geocode(address)
sw = [47.212427, 38.925331]
ne = [47.286151, 38.959084]
lat = ne[0]
lon = ne[1]
search_url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&intent=browse&sw={},{}&ne={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            #address,
            lat,
            lon,
            lat,
            lon,
            500)
search_url

'https://api.foursquare.com/v2/venues/search?&client_id=V53MHPBBL23EVUFTF31HTKJAFRJIK2QL1WJURTZ3ARRFF3KE&client_secret=G5Z0ALDAAKEOPESPXKBZPXNB2TIEJKGWM33ZGSBCNW0VGW0T&v=20180604&intent=browse&sw=47.286151,38.959084&ne=47.286151,38.959084&limit=500'

In [50]:
results = requests.get(search_url).json()

In [51]:
json_to_file(results)